# Kelvin unit: compliance function & stress redistribution

***

<center><i>Petr Havlásek (c) 2023-24, petr.havlasek@cvut.cz</i></center>

***


In [1]:
online = False

if (online):
    import micropip
    await micropip.install('ipywidgets')

import math
import numpy as np    
    
import matplotlib.pyplot as plt
    
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

from IPython.display import display


**Notation:**

>`E` ... spring stiffness [MPa]<br>

>`eta` ... dashpot viscosity [MPa day]<br>

>`tau` ... retardation time [day] (=`eta/E`)<br>

>`t` ... time of interest [day]<br>

>`tt` ... time of loading [day]<br>

## Definition of a compliance function for Kelvin unit

In [2]:
def Kelvin_J_func(E, eta, t, tt):

    tau = eta / E

    if t > tt:
        dt = t-tt

        if tau > 0.:
            return 1./ E * ( 1. - math.exp(-(dt)/ tau) )
        else:
            return 1./ E
            
    else:
        return 0.
   

## Parameters definition

In [3]:
# reference spring stiffness [MPa]
E_ref = 1.
# reference dashpot viscosity [MPa * day]
eta_ref = 1.
# reference retardation time
tau_ref = eta_ref / E_ref

# variables ranges: powers of 10
# stiffness
log_E_min = -3
log_E_max = 3

# viscosity
log_eta_min = -3
log_eta_max = 3

log_tau_min = -2
log_tau_max = 2

# number of time steps
t_div = 100

## Compliance function

In [4]:

def plot_Kelvin_J_func(E, eta, log_scale = True, show_tau = False, show_reference = True):
    
    if (log_scale):
        times = np.logspace(log_tau_min, log_tau_max, num = t_div )
    else:
        times = np.linspace(0., 10**log_tau_max, num = t_div )

    ax.clear()
        
    # reference data with E_ref and eta_ref
    if show_reference:
        J_ref = []
        for t in times:
            J_ref.append( Kelvin_J_func(E_ref, eta_ref, t, 0.) )

        ax.plot(times, J_ref, lw=1.5, color="red", linestyle='dashed', label='Jref(t)')
        
        if (show_tau):
            ax.axvline(x = tau_ref, lw=1.5, color="red", linestyle = "--")

    # data for user-defined parameters
    J = []
    for t in times:
        J.append( Kelvin_J_func(E, eta, t, 0.) )
    ax.plot(times, J, lw=2., color="blue", label=r'J(t)')
    if (show_tau):
        ax.axvline(x = eta/E, lw=1.5, color="blue", linestyle = "--")
    
    if (log_scale):
        ax.set_xscale('log')
        ax.set_xlim([10**log_tau_min, 10**log_tau_max])
    else:
        ax.set_xlim([0., 10**log_tau_max])


    ax.grid(True)
    ax.legend()
    ax.set_xlabel('Duration of loading, t-t\' [day]')
    ax.set_ylabel('Compliance, J [1/MPa]')  
    
    display(fig)

### GUI for compliance function

In [5]:


output = widgets.Output()

fig, ax = plt.subplots(1, 1, figsize=(10,5))
plt.rcParams.update({'font.size': 14})
plt.close(fig)


E_slide = widgets.FloatLogSlider(min=log_E_min, max=log_E_max, value=1., step=0.25, description='E [MPa]')
eta_slide = widgets.FloatLogSlider(min=log_eta_min, max=log_eta_max, value=1., step=0.25, description='eta [MPa day]')

aux = interactive(plot_Kelvin_J_func, E = E_slide, eta = eta_slide, log_scale = False, show_tau = False, show_reference = True)

display(aux)


interactive(children=(FloatLogSlider(value=1.0, description='E [MPa]', max=3.0, min=-3.0, step=0.25), FloatLog…

## Stress redistribution under constant stress level

In [6]:
def plot_Kelvin_stresses(E, eta, sigma, log_scale = True, show_tau = False, show_reference = True):

    if (log_scale):
        times = np.logspace(log_tau_min, log_tau_max, num = t_div )
    else:
        times = np.linspace(0., 10**log_tau_max, num = t_div )
        
    ax2.clear()        

    # reference data with E_ref and eta_ref
    if show_reference:
        sig_e_ref = []
        sig_v_ref = []
        for t in times:
            J_ref = Kelvin_J_func(E_ref, eta_ref, t, 0.)
            eps_ref = sigma * J_ref
            sig_e_ref.append( E_ref * eps_ref )
            sig_v_ref.append( sigma - E_ref * eps_ref )

        ax2.plot(times, sig_e_ref, lw=1.5, color="red", linestyle='dashed', label='sig_e_ref(t)')
        ax2.plot(times, sig_v_ref, lw=1.5, color="red", linestyle='solid', label='sig_v_ref(t)')
        
        if (show_tau):
            ax2.axvline(x = tau_ref, lw=1.5, color="red", linestyle = "--")

    # data for user-defined parameters
    sig_e = []
    sig_v = []
    for t in times:
        J = Kelvin_J_func(E, eta, t, 0.)
        eps = sigma * J
        sig_e.append( E * eps )
        sig_v.append( sigma - E * eps )


    ax2.plot(times, sig_e, lw=2.0, color="blue", linestyle='dashed', label='sig_e(t)')
    ax2.plot(times, sig_v, lw=2.0, color="blue", linestyle='solid', label='sig_v(t)')
    
    if (show_tau):
        ax2.axvline(x = eta/E, lw=1.5, color="blue", linestyle = "--")
    
    if (log_scale):
        ax2.set_xscale('log')
        ax2.set_xlim([10**log_tau_min, 10**log_tau_max])
    else:
        ax2.set_xlim([0., 10**log_tau_max])


    ax2.grid(True)
    ax2.legend()
    ax2.set_xlabel('Duration of loading, t-t\' [day]')
    ax2.set_ylabel('Stress, sigma [MPa]')
    
    display(fig2)


## GUI for stress redistribution

In [7]:
output = widgets.Output()

fig2, ax2 = plt.subplots(1, 1, figsize=(10,5))
plt.rcParams.update({'font.size': 14})
plt.close(fig2)


E_slide = widgets.FloatLogSlider(min=log_E_min, max=log_E_max, value=1., step=0.25, description='E [MPa]')
eta_slide = widgets.FloatLogSlider(min=log_eta_min, max=log_eta_max, value=1., step=0.25, description='eta [MPa day]')
sigma_slide = widgets.FloatSlider(min=0., max=100., value=10., step=1., description='sigma [MPa]')

aux = interactive(plot_Kelvin_stresses, E = E_slide, eta = eta_slide, sigma = sigma_slide, log_scale = False, show_tau = False, show_reference = True)

display(aux)


interactive(children=(FloatLogSlider(value=1.0, description='E [MPa]', max=3.0, min=-3.0, step=0.25), FloatLog…